## Test with straight beam

The aim of this test is to compare the inertial loads of an analytical case with the ones given by the code.

The beam used is straight, measures 1 meter, weighs 100 kg and it only has two nodes. It is defined with the following half chord position and property files.


The stiffness matrix is not needed in this test, but, to make the structural code work, a valid matrix was used from the first node of the original IEA 15 MW reference WT file.

In [265]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from beam_corot.ComplBeam import ComplBeam
from beam_corot.CoRot import CoRot
from utils import save_load
from inertial_forces import b1_to_b2, pos_b1_to_b2, inertial_loads_fun, inertial_loads_fun_v02, inertial_loads_fun_v03,inertial_loads_fun_v04
from cg_offset import get_cg_offset

In [266]:
# Model input json file name
f_model_json = "straight_beam_no_offset_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_no_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_no_offset.nodeLocations[:,2] # z-axis position
beam_no_offset.nodeLocations

pos = np.concatenate((beam_no_offset.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

-----------
ComplBeam Model Created
Static analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])


In [267]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_no_offset_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_no_offset_dyn = ComplBeam(mainfile_dyn)
M_mat_no_offset = beam_no_offset_dyn.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_no_offset_df = pd.DataFrame(M_mat_no_offset)
M_mat_no_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,0.000000,0.042634,4.209301,-0.093673,16.499820,0.207198,0.000000,0.042634,-4.124033,0.093673
1,-0.207198,33.580883,0.000000,-4.229476,-0.062810,-0.137028,0.207198,16.419117,0.000000,4.103857,-0.062810,0.137028
2,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000,0.000000,0.000000,16.666667,0.000000,0.000000,0.000000
3,0.042634,-4.229476,0.000000,0.834256,0.000922,0.000751,-0.042634,-4.103857,0.000000,-0.832411,0.000922,-0.000751
4,4.209301,-0.062810,0.000000,0.000922,0.834256,0.000751,4.124033,0.062810,0.000000,0.000922,-0.832411,-0.000751
5,-0.093673,-0.137028,0.000000,0.000751,0.000751,0.004702,0.093673,0.137028,0.000000,0.000751,0.000751,-0.004702
6,16.499820,0.207198,0.000000,-0.042634,4.124033,0.093673,33.500180,-0.207198,0.000000,-0.042634,-4.209301,-0.093673
7,0.207198,16.419117,0.000000,-4.103857,0.062810,0.137028,-0.207198,33.580883,0.000000,4.229476,0.062810,-0.137028
8,0.000000,0.000000,16.666667,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000
9,0.042634,4.103857,0.000000,-0.832411,0.000922,0.000751,-0.042634,4.229476,0.000000,0.834256,0.000922,-0.000751


## Test 1
No CoG offset, no pitch angle, no hub radius.

The operational condicions are:

    RPM = 1 rad/s
    pitch = 0

The centrifugar force in its center of mass is $F=m\cdot \omega^{2}\cdot r= 100 \cdot 1 \cdot 0.5 = 50 \text{ N}$.

In [268]:
# Operations info
omega = 1 # rad/s

In [269]:
# Calculate inertial loads using the CoG accelerations (positions in our input)
offset = np.zeros((2,3))

load_t1 = inertial_loads_fun_v03(pos,offset,M_mat_no_offset,0,omega,0)
load_t1 = np.reshape(load_t1, (-1,6))

force, moment = np.split(load_t1,[3],axis=1)
force_df = pd.DataFrame(force, columns=['X', 'Y', 'Z'])
force_df


,X,Y,Z
0,0.0,0.0,-16.666667
1,0.0,0.0,-33.333333


In [270]:
moment_df = pd.DataFrame(moment, columns=['X', 'Y', 'Z'])
moment_df

,X,Y,Z
0,0.0,0.0,0.0
1,0.0,0.0,0.0


In [271]:
force, moment = np.split(load_t1,[3],axis=1)
force_o = np.sum(force, axis=0)
print(f'{force_o = }')

r_globcg = np.array([0,0,0.5]) # The global CoG
r_globcg2node = pos[:,:3] + offset - r_globcg
print(f'{r_globcg2node = }')

moment_o = np.sum(moment,axis=0) + np.cross(r_globcg2node[0],force[0]) + np.cross(r_globcg2node[1],force[1])
print(f'{moment_o = }')

force_o = array([  0.,   0., -50.])
r_globcg2node = array([[ 0. ,  0. , -0.5],
       [ 0. ,  0. ,  0.5]])
moment_o = array([0., 0., 0.])


In [272]:
f_analytical = np.array([0,0,-50])
m_analytical = np.zeros(3)

if np.allclose(force_o, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_o, m_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_o, f_analytical) and np.allclose(moment_o, m_analytical):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
The two results are the same.
Test passed.


## Test 1.5

This time the beam is 2 meters long.

In [273]:
# Model input json file name
f_model_json = "straight_beam_2m_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_no_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_no_offset.nodeLocations[:,2] # z-axis position
beam_no_offset.nodeLocations

pos = np.concatenate((beam_no_offset.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

-----------
ComplBeam Model Created
Static analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 0., 0., 0.]])


In [274]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_2m_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_no_offset_dyn = ComplBeam(mainfile_dyn)
M_mat_no_offset = beam_no_offset_dyn.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_no_offset_df = pd.DataFrame(M_mat_no_offset)
M_mat_no_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,67.685220,-1.254818,0.000000,0.540898,17.207565,-0.765513,32.314780,1.254818,0.000000,0.540898,-16.125768,0.765513
1,-1.254818,68.157749,0.000000,-17.443829,-0.777163,-1.062936,1.254818,31.842251,0.000000,15.889504,-0.777163,1.062936
2,0.000000,0.000000,66.666667,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000
3,0.540898,-17.443829,0.000000,6.729910,0.063243,0.030580,-0.540898,-15.889504,0.000000,-6.603424,0.063243,-0.030580
4,17.207565,-0.777163,0.000000,0.063243,6.729910,0.030580,16.125768,0.777163,0.000000,0.063243,-6.603424,-0.030580
5,-0.765513,-1.062936,0.000000,0.030580,0.030580,0.145673,0.765513,1.062936,0.000000,0.030580,0.030580,-0.145673
6,32.314780,1.254818,0.000000,-0.540898,16.125768,0.765513,67.685220,-1.254818,0.000000,-0.540898,-17.207565,-0.765513
7,1.254818,31.842251,0.000000,-15.889504,0.777163,1.062936,-1.254818,68.157749,0.000000,17.443829,0.777163,-1.062936
8,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000,0.000000,0.000000,66.666667,0.000000,0.000000,0.000000
9,0.540898,15.889504,0.000000,-6.603424,0.063243,0.030580,-0.540898,17.443829,0.000000,6.729910,0.063243,-0.030580


In [275]:
# Calculate inertial loads using the CoG accelerations (positions in our input)
offset = np.zeros((2,3))

load_t1 = inertial_loads_fun_v03(pos,offset,M_mat_no_offset,0,omega,0)
load_t1 = np.reshape(load_t1, (-1,6))

force, moment = np.split(load_t1,[3],axis=1)
print(f'{force = }')
print(f'{moment = }')

force = array([[   0.        ,    0.        ,  -66.66666667],
       [   0.        ,    0.        , -133.33333333]])
moment = array([[0., 0., 0.],
       [0., 0., 0.]])


In [276]:
force, moment = np.split(load_t1,[3],axis=1)
force_o = np.sum(force, axis=0)
print(f'{force_o = }')

r_globcg = np.array([0,0,1]) # The global CoG
r_globcg2node = pos[:,:3] + offset - r_globcg
print(f'{r_globcg2node = }')

moment_o = np.sum(moment,axis=0) + np.cross(r_globcg2node[0],force[0]) + np.cross(r_globcg2node[1],force[1])
print(f'{moment_o = }')

force_o = array([   0.,    0., -200.])
r_globcg2node = array([[ 0.,  0., -1.],
       [ 0.,  0.,  1.]])
moment_o = array([0., 0., 0.])


In [277]:
f_analytical = np.array([0,0,-200])
m_analytical = np.zeros(3)

if np.allclose(force_o, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_o, m_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_o, f_analytical) and np.allclose(moment_o, m_analytical):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
The two results are the same.
Test passed.


## Test 2

There is a 10 cm offset of the CoG in the x-direction.

No hub, no pitch

In [278]:
# Model input json file name
f_model_json = "straight_beam_offset_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_offset.nodeLocations[:,2] # z-axis position
beam_offset.nodeLocations

pos = np.concatenate((beam_offset.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

-----------
ComplBeam Model Created
Static analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])


In [279]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_offset_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_offset_dyn = ComplBeam(mainfile_dyn)
M_mat_offset = beam_offset_dyn.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_offset_df = pd.DataFrame(M_mat_offset)
M_mat_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,0.244735,0.042634,4.209301,-0.105866,16.499820,0.207198,0.244735,0.042634,-4.124033,0.105866
1,-0.207198,33.580883,-0.244735,-4.229476,-0.062810,3.208498,0.207198,16.419117,-0.244735,4.103857,-0.062810,1.791502
2,0.244735,-0.244735,33.333333,0.122368,-3.210966,-0.285525,-0.244735,0.244735,16.666667,0.122368,-1.544299,0.285525
3,0.042634,-4.229476,0.122368,0.834256,0.000922,-0.422012,-0.042634,-4.103857,0.122368,-0.832411,0.000922,-0.411321
4,4.209301,-0.062810,-3.210966,0.000922,0.834256,-0.005346,4.124033,0.062810,-1.544299,0.000922,-0.832411,0.005346
5,-0.105866,3.208498,-0.285525,-0.422012,-0.005346,-0.022404,0.105866,1.791502,-0.285525,0.411321,-0.005346,0.022404
6,16.499820,0.207198,-0.244735,-0.042634,4.124033,0.105866,33.500180,-0.207198,-0.244735,-0.042634,-4.209301,-0.105866
7,0.207198,16.419117,0.244735,-4.103857,0.062810,1.791502,-0.207198,33.580883,0.244735,4.229476,0.062810,3.208498
8,0.244735,-0.244735,16.666667,0.122368,-1.544299,-0.285525,-0.244735,0.244735,33.333333,0.122368,-3.210966,0.285525
9,0.042634,4.103857,0.122368,-0.832411,0.000922,0.411321,-0.042634,4.229476,0.122368,0.834256,0.000922,0.422012


In [280]:
# Calculate inertial loads using the CoG accelerations
offset = get_cg_offset(beam_offset)
print(f'{offset = }')

load_t2 = inertial_loads_fun_v04(pos,offset,M_mat_offset,0,omega,0)
load_t2 = np.reshape(load_t2, (-1,6))

force, moment = np.split(load_t2,[3],axis=1)
# print(f'{force = }')
# print(f'{moment = }')

offset = array([[0.1, 0. , 0. ],
       [0.1, 0. , 0. ]])


In [281]:
force_df = pd.DataFrame(force, columns=['X', 'Y', 'Z'])
print('force')
force_df

force


,X,Y,Z
0,-5.244735,0.244735,-16.666667
1,-4.755265,-0.244735,-33.333333


In [282]:
moment_df = pd.DataFrame(moment, columns=['X', 'Y', 'Z'])
print('moment')
moment_df

moment


,X,Y,Z
0,-0.122368,0.710966,0.285525
1,-0.122368,4.044299,-0.285525


In [283]:
force, moment = np.split(load_t2,[3],axis=1)
force_o = np.sum(force, axis=0)
print(f'{force_o = }')

r_globcg = np.array([0.1,0,0.5]) # The global CoG
r_globcg2node = pos[:,:3] - r_globcg
print(f'{r_globcg2node = }')

moment_o = np.sum(moment,axis=0) + np.cross(r_globcg2node[0],force[0]) + np.cross(r_globcg2node[1],force[1])
print(f'{moment_o = }')
print(np.sum(moment,axis=0))
print(np.cross(r_globcg2node[0],force[0]) + np.cross(r_globcg2node[1],force[1]))

force_o = array([-1.00000000e+01, -2.77555756e-17, -5.00000000e+01])
r_globcg2node = array([[-0.1,  0. , -0.5],
       [-0.1,  0. ,  0.5]])
moment_o = array([5.20278265e-14, 8.53539461e-13, 3.46944695e-18])
[-0.24473539  4.75526461  0.        ]
[ 2.44735387e-01 -4.75526461e+00  3.46944695e-18]


In [284]:
f_analytical = np.array([-10,0.,-50])
m_analytical = np.zeros(3)

if np.allclose(force_o, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_o, m_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_o, f_analytical) and np.allclose(moment_o, m_analytical):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
The two results are the same.
Test passed.


In [285]:
force, moment = np.split(load_t2,[3],axis=1)
force_o = np.sum(force, axis=0)
print(f'{force_o = }')

moment_o = np.sum(moment,axis=0) + np.cross(pos[0,:3],force[0]) + np.cross(pos[1,:3],force[1])
print(f'{moment_o = }')

force_o = array([-1.00000000e+01, -2.77555756e-17, -5.00000000e+01])
moment_o = array([5.20417043e-14, 8.52651283e-13, 0.00000000e+00])


## Test 3
The pitch is 90 degrees. There is no hub radius.

The offset of the CoG is now in the y-axis of the blade coordinate system.

In [286]:
# Model input json file name
f_model_json = "straight_beam_offset_y_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_offset.nodeLocations[:,2] # z-axis position
beam_offset.nodeLocations

pos = np.concatenate((beam_offset.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

-----------
ComplBeam Model Created
Static analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])


In [287]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_offset_y_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_offset = ComplBeam(mainfile_dyn)
M_mat_offset = beam_offset.M_mat_full
pos = np.concatenate((beam_offset.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

# Display DataFrame in Jupyter notebook
M_mat_offset_df = pd.DataFrame(M_mat_offset)
M_mat_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,-0.365790,0.042634,4.209301,-3.435164,16.499820,0.207198,-0.365790,0.042634,-4.124033,-1.564836
1,-0.207198,33.580883,0.365790,-4.229476,-0.062810,-0.128870,0.207198,16.419117,0.365790,4.103857,-0.062810,0.128870
2,-0.365790,0.365790,33.333333,3.150439,-0.182895,-0.406579,0.365790,-0.365790,16.666667,1.483772,-0.182895,0.406579
3,0.042634,-4.229476,3.150439,0.834256,0.000922,-0.003328,-0.042634,-4.103857,1.483772,-0.832411,0.000922,0.003328
4,4.209301,-0.062810,-0.182895,0.000922,0.834256,-0.419995,4.124033,0.062810,-0.182895,0.000922,-0.832411,-0.413339
5,-3.435164,-0.128870,-0.406579,-0.003328,-0.419995,0.023737,-1.564836,0.128870,-0.406579,-0.003328,0.413339,-0.023737
6,16.499820,0.207198,0.365790,-0.042634,4.124033,-1.564836,33.500180,-0.207198,0.365790,-0.042634,-4.209301,-3.435164
7,0.207198,16.419117,-0.365790,-4.103857,0.062810,0.128870,-0.207198,33.580883,-0.365790,4.229476,0.062810,-0.128870
8,-0.365790,0.365790,16.666667,1.483772,-0.182895,-0.406579,0.365790,-0.365790,33.333333,3.150439,-0.182895,0.406579
9,0.042634,4.103857,1.483772,-0.832411,0.000922,-0.003328,-0.042634,4.229476,3.150439,0.834256,0.000922,0.003328


In [288]:
offset = get_cg_offset(beam_offset)
print(f'{offset = }')

# Operations info
omega = 1 # rad/s
pitch_deg = 90
pitch_rad = np.deg2rad(pitch_deg)

r_hub = 0

offset = array([[0. , 0.1, 0. ],
       [0. , 0.1, 0. ]])


In [289]:
# Calculate inertial loads using the CoG accelerations
offset = np.array([[0,0.1,0],[0,0.1,0]])

load_t3 = inertial_loads_fun_v04(pos,offset,M_mat_offset,0,omega,pitch_rad)
load_t3 = np.reshape(load_t3, (-1,6))

force, moment = np.split(load_t3,[3],axis=1)

In [290]:
force, moment = np.split(load_t3,[3],axis=1)
force_o = np.sum(force, axis=0)
print(f'{force_o = }')

pos_o = pos.copy()

moment_o = np.sum(moment,axis=0) + np.cross(pos[0,:3],force[0]) + np.cross(pos[1,:3],force[1])
print(f'{moment_o = }')

force_o = array([ 6.10622664e-16, -1.00000000e+01, -5.00000000e+01])
moment_o = array([ 6.1284311e-13, -2.8255176e-14,  0.0000000e+00])


In [291]:
f_analytical = np.array([0.,-10.,-50])
m_analytical = np.zeros(3)

if np.allclose(force_o, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_o, m_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_o, f_analytical) and np.allclose(moment_o, m_analytical):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
The two results are the same.
Test passed.


## Test 4

Adding hub radius of 1 meter. No pitch, but offset.

In [292]:
# Model input json file name
f_model_json = "straight_beam_offset_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_offset.nodeLocations[:,2] # z-axis position
beam_offset.nodeLocations

pos = np.concatenate((beam_offset.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

-----------
ComplBeam Model Created
Static analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])


In [293]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_offset_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_offset_dyn = ComplBeam(mainfile_dyn)
M_mat_offset = beam_offset_dyn.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_offset_df = pd.DataFrame(M_mat_offset)
M_mat_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,0.244735,0.042634,4.209301,-0.105866,16.499820,0.207198,0.244735,0.042634,-4.124033,0.105866
1,-0.207198,33.580883,-0.244735,-4.229476,-0.062810,3.208498,0.207198,16.419117,-0.244735,4.103857,-0.062810,1.791502
2,0.244735,-0.244735,33.333333,0.122368,-3.210966,-0.285525,-0.244735,0.244735,16.666667,0.122368,-1.544299,0.285525
3,0.042634,-4.229476,0.122368,0.834256,0.000922,-0.422012,-0.042634,-4.103857,0.122368,-0.832411,0.000922,-0.411321
4,4.209301,-0.062810,-3.210966,0.000922,0.834256,-0.005346,4.124033,0.062810,-1.544299,0.000922,-0.832411,0.005346
5,-0.105866,3.208498,-0.285525,-0.422012,-0.005346,-0.022404,0.105866,1.791502,-0.285525,0.411321,-0.005346,0.022404
6,16.499820,0.207198,-0.244735,-0.042634,4.124033,0.105866,33.500180,-0.207198,-0.244735,-0.042634,-4.209301,-0.105866
7,0.207198,16.419117,0.244735,-4.103857,0.062810,1.791502,-0.207198,33.580883,0.244735,4.229476,0.062810,3.208498
8,0.244735,-0.244735,16.666667,0.122368,-1.544299,-0.285525,-0.244735,0.244735,33.333333,0.122368,-3.210966,0.285525
9,0.042634,4.103857,0.122368,-0.832411,0.000922,0.411321,-0.042634,4.229476,0.122368,0.834256,0.000922,0.422012


In [294]:
offset = get_cg_offset(beam_offset)
print(f'{offset = }')

# Operations info
omega = 1 # rad/s
pitch_deg = 0
pitch_rad = np.deg2rad(pitch_deg)

r_hub = 1

offset = array([[0.1, 0. , 0. ],
       [0.1, 0. , 0. ]])


In [295]:
# Calculate inertial loads using the CoG accelerations

load_t4 = inertial_loads_fun_v04(pos,offset,M_mat_offset,r_hub,omega,0)
load_t4 = np.reshape(load_t4, (-1,6))
print(f'{pos = }')

force, moment = np.split(load_t4,[3],axis=1)
print(f'{force = }')
print(f'{moment = }')

pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])
force = array([[ -5.73420616,   0.73420616, -66.66666667],
       [ -4.26579384,  -0.73420616, -83.33333333]])
moment = array([[-0.36710308,  5.46623025,  0.85657386],
       [-0.36710308,  8.79956359, -0.85657386]])


In [296]:
force_o = np.sum(force, axis=0)
print(f'{force_o = }')

pos_o = pos[:,:3].copy()
pos_o[:,2] += r_hub
print(f'{pos_o = }')

# pos_o = np.reshape(pos_b1_to_b2(pos, 0, r_hub, inv=True),(-1,6))[:,:3]
# print(f'{pos_o = }')

moment_o = np.sum(moment,axis=0) + np.cross(pos_o[0],force[0]) + np.cross(pos_o[1],force[1])
print(f'{moment_o = }')

force_o = array([-1.00000000e+01, -1.11022302e-16, -1.50000000e+02])
pos_o = array([[0., 0., 1.],
       [0., 0., 2.]])
moment_o = array([-3.25961480e-13,  1.20259358e-12,  1.11022302e-16])


In [297]:
f_analytical = np.array([-10.,0.,-150.])
m_analytical = np.zeros(3)

if np.allclose(force_o, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_o, m_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_o, f_analytical) and np.allclose(moment_o, m_analytical):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
The two results are the same.
Test passed.


## Test 5

Let's use a 2 meter long straight beam.
No offset, no pitch, no hub.

In [298]:
# Model input json file name
f_model_json = "straight_beam_2m_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_offset.nodeLocations[:,2] # z-axis position
beam_offset.nodeLocations

pos = np.concatenate((beam_offset.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

-----------
ComplBeam Model Created
Static analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 0., 0., 0.]])


In [299]:

# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_2m_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_offset_dyn = ComplBeam(mainfile_dyn)
M_mat_offset = beam_offset_dyn.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_offset_df = pd.DataFrame(M_mat_offset)
M_mat_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,67.685220,-1.254818,0.000000,0.540898,17.207565,-0.765513,32.314780,1.254818,0.000000,0.540898,-16.125768,0.765513
1,-1.254818,68.157749,0.000000,-17.443829,-0.777163,-1.062936,1.254818,31.842251,0.000000,15.889504,-0.777163,1.062936
2,0.000000,0.000000,66.666667,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000
3,0.540898,-17.443829,0.000000,6.729910,0.063243,0.030580,-0.540898,-15.889504,0.000000,-6.603424,0.063243,-0.030580
4,17.207565,-0.777163,0.000000,0.063243,6.729910,0.030580,16.125768,0.777163,0.000000,0.063243,-6.603424,-0.030580
5,-0.765513,-1.062936,0.000000,0.030580,0.030580,0.145673,0.765513,1.062936,0.000000,0.030580,0.030580,-0.145673
6,32.314780,1.254818,0.000000,-0.540898,16.125768,0.765513,67.685220,-1.254818,0.000000,-0.540898,-17.207565,-0.765513
7,1.254818,31.842251,0.000000,-15.889504,0.777163,1.062936,-1.254818,68.157749,0.000000,17.443829,0.777163,-1.062936
8,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000,0.000000,0.000000,66.666667,0.000000,0.000000,0.000000
9,0.540898,15.889504,0.000000,-6.603424,0.063243,0.030580,-0.540898,17.443829,0.000000,6.729910,0.063243,-0.030580


In [300]:
offset = get_cg_offset(beam_offset)
print(f'{offset = }')

# Operations info
omega = 1 # rad/s
pitch_deg = 0
pitch_rad = np.deg2rad(pitch_deg)

r_hub = 0

offset = array([[0., 0., 0.],
       [0., 0., 0.]])


In [301]:
# Calculate inertial loads using the CoG accelerations

load_t5 = inertial_loads_fun_v04(pos,offset,M_mat_offset,r_hub,omega,0)
load_t5 = np.reshape(load_t5, (-1,6))
print(f'{pos = }')

force, moment = np.split(load_t5,[3],axis=1)
print(f'{force = }')
print(f'{moment = }')

pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 0., 0., 0.]])
force = array([[   0.        ,    0.        ,  -66.66666667],
       [   0.        ,    0.        , -133.33333333]])
moment = array([[0., 0., 0.],
       [0., 0., 0.]])


In [302]:
force_o = np.sum(force, axis=0)
print(f'{force_o = }')

pos_o = pos[:,:3].copy()
pos_o[:,2] += r_hub
print(f'{pos_o = }')

moment_o = np.sum(moment,axis=0) + np.cross(pos_o[0],force[0]) + np.cross(pos_o[1],force[1])
print(f'{moment_o = }')

force_o = array([   0.,    0., -200.])
pos_o = array([[0., 0., 0.],
       [0., 0., 2.]])
moment_o = array([0., 0., 0.])


In [303]:
f_analytical = np.array([0.,0.,-200])
m_analytical = np.zeros(3)

if np.allclose(force_o, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_o, m_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_o, f_analytical) and np.allclose(moment_o, m_analytical):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
The two results are the same.
Test passed.


## Test 6

CoG offset and more than two nodes.

Length 2 meters, no hub, no pitch.

In [304]:
# Model input json file name
f_model_json = "straight_beam_offset_3nodes_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_offset.nodeLocations[:,2] # z-axis position
beam_offset.nodeLocations

pos = np.concatenate((beam_offset.nodeLocations,np.zeros((beam_offset.nodeLocations.shape[0],3))), axis=1)
print(f'{pos = }')

-----------
ComplBeam Model Created
Static analysis done
pos = array([[0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. ]])


In [305]:

# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_offset_3nodes_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_offset_dyn = ComplBeam(mainfile_dyn)
M_mat_offset = beam_offset_dyn.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_offset_df = pd.DataFrame(M_mat_offset)
M_mat_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


/home/juliovmlo/master_thesis/master_thesis/beam_corot/ComplBeam.py:752: ComplexWarning: Casting complex values to real discards the imaginary part
  self.eigen_vec_full[self.dofs_free, :] = self.eigen_vec


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,16.689257,-0.028143,0.067352,0.002841,1.044508,-0.013317,8.310743e+00,2.814274e-02,6.735184e-02,2.841255e-03,-1.038825e+00,1.331660e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.028143,16.700362,-0.067352,-1.045896,-0.004229,1.651045,2.814274e-02,8.299638e+00,-6.735184e-02,1.037437e+00,-4.229357e-03,8.489552e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.067352,-0.067352,16.666667,0.016838,-1.649829,-0.070158,-6.735184e-02,6.735184e-02,8.333333e+00,1.683796e-02,-8.164954e-01,7.015817e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.002841,-1.045896,0.016838,0.104175,0.000009,-0.104573,-2.841255e-03,-1.037437e+00,1.683796e-02,-1.041579e-01,8.732127e-06,-1.037608e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1.044508,-0.004229,-1.649829,0.000009,0.104175,-0.000406,1.038825e+00,4.229357e-03,-8.164954e-01,8.732127e-06,-1.041579e-01,4.058931e-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,-0.013317,1.651045,-0.070158,-0.104573,-0.000406,-0.003300,1.331660e-02,8.489552e-01,-7.015817e-02,1.037608e-01,-4.058931e-04,3.300242e-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,8.310743,0.028143,-0.067352,-0.002841,1.038825,0.013317,3.337851e+01,-5.628547e-02,1.913192e-13,-1.920959e-12,-1.315614e-12,-2.663320e-02,8.310743,0.028143,0.067352,0.002841,-1.038825,0.013317
7,0.028143,8.299638,0.067352,-1.037437,0.004229,0.848955,-5.628547e-02,3.340072e+01,-1.913192e-13,-7.041034e-13,-1.083005e-12,3.302090e+00,0.028143,8.299638,-0.067352,1.037437,-0.004229,0.848955
8,0.067352,-0.067352,8.333333,0.016838,-0.816495,-0.070158,1.912498e-13,-1.912498e-13,3.333333e+01,3.367592e-02,-3.299657e+00,2.223222e-14,-0.067352,0.067352,8.333333,0.016838,-0.816495,0.070158
9,0.002841,1.037437,0.016838,-0.104158,0.000009,0.103761,-1.920925e-12,-7.045475e-13,3.367592e-02,2.083508e-01,1.746425e-05,-6.322720e-14,-0.002841,-1.037437,0.016838,-0.104158,0.000009,-0.103761


In [306]:
offset = get_cg_offset(beam_offset)
print(f'{offset = }')

# Operations info
omega = 1 # rad/s
pitch_deg = 0
pitch_rad = np.deg2rad(pitch_deg)

r_hub = 0

offset = array([[0.1, 0. , 0. ],
       [0.1, 0. , 0. ],
       [0.1, 0. , 0. ]])


In [307]:
# Calculate inertial loads using the CoG accelerations

load_t6 = inertial_loads_fun_v04(pos,offset,M_mat_offset,r_hub,omega,0)
load_t6 = np.reshape(load_t6, (-1,6))
print(f'{pos = }')

force, moment = np.split(load_t6,[3],axis=1)
print(f'{force = }')
print(f'{moment = }')

pos = array([[0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. ]])
force = array([[ -2.53367592,   0.03367592,  -4.16666667],
       [ -5.06735184,   0.06735184, -25.        ],
       [ -2.39897223,  -0.10102777, -20.83333333]])
moment = array([[-0.00841898,  0.19991435,  0.03507909],
       [-0.03367592,  2.46632408,  0.07015817],
       [-0.02525694,  2.26640973, -0.10523726]])


In [308]:
force_o = np.sum(force, axis=0)
print(f'{force_o = }')

pos_o = pos[:,:3].copy()
pos_o[:,2] += r_hub
print(f'{pos_o = }')

moment_o = np.sum(moment,axis=0)
for node_i in range(pos_o.shape[0]):
    moment_o += np.cross(pos_o[node_i],force[node_i])
print(f'{moment_o = }')

force_o = array([-1.00000000e+01,  1.38777878e-17, -5.00000000e+01])
pos_o = array([[0. , 0. , 0. ],
       [0. , 0. , 0.5],
       [0. , 0. , 1. ]])
moment_o = array([ 6.24847396e-13, -2.06945572e-13,  0.00000000e+00])


In [309]:
f_analytical = np.array([-10.,0.,-50])
m_analytical = np.zeros(3)

if np.allclose(force_o, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_o, m_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_o, f_analytical) and np.allclose(moment_o, m_analytical):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
The two results are the same.
Test passed.


## Test 7

- CoG offset: 10 cm in the x-axis
- pitch angle: 45 degrees
- hub radius: 1 meter

The analitycal result is:

$p_{cg,B1} = (0.1\cdot\sqrt 2 /2, 0.1\cdot\sqrt 2 /2, 1.5)$

$m = 100 kg$

$F_{B1} = (-m\omega^2\cdot u_x, 0, -m\omega^2\cdot u_z) = (-5\cdot\sqrt 2 , 0, -150)$

In [310]:
# Model input json file name
f_model_json = "straight_beam_offset_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_offset.nodeLocations[:,2] # z-axis position
beam_offset.nodeLocations

pos = np.concatenate((beam_offset.nodeLocations,np.zeros((beam_offset.nodeLocations.shape[0],3))), axis=1)
print(f'{pos = }')


-----------
ComplBeam Model Created
Static analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])


In [311]:

# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_offset_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_offset_dyn = ComplBeam(mainfile_dyn)
M_mat_offset = beam_offset_dyn.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_offset_df = pd.DataFrame(M_mat_offset)
M_mat_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,0.244735,0.042634,4.209301,-0.105866,16.499820,0.207198,0.244735,0.042634,-4.124033,0.105866
1,-0.207198,33.580883,-0.244735,-4.229476,-0.062810,3.208498,0.207198,16.419117,-0.244735,4.103857,-0.062810,1.791502
2,0.244735,-0.244735,33.333333,0.122368,-3.210966,-0.285525,-0.244735,0.244735,16.666667,0.122368,-1.544299,0.285525
3,0.042634,-4.229476,0.122368,0.834256,0.000922,-0.422012,-0.042634,-4.103857,0.122368,-0.832411,0.000922,-0.411321
4,4.209301,-0.062810,-3.210966,0.000922,0.834256,-0.005346,4.124033,0.062810,-1.544299,0.000922,-0.832411,0.005346
5,-0.105866,3.208498,-0.285525,-0.422012,-0.005346,-0.022404,0.105866,1.791502,-0.285525,0.411321,-0.005346,0.022404
6,16.499820,0.207198,-0.244735,-0.042634,4.124033,0.105866,33.500180,-0.207198,-0.244735,-0.042634,-4.209301,-0.105866
7,0.207198,16.419117,0.244735,-4.103857,0.062810,1.791502,-0.207198,33.580883,0.244735,4.229476,0.062810,3.208498
8,0.244735,-0.244735,16.666667,0.122368,-1.544299,-0.285525,-0.244735,0.244735,33.333333,0.122368,-3.210966,0.285525
9,0.042634,4.103857,0.122368,-0.832411,0.000922,0.411321,-0.042634,4.229476,0.122368,0.834256,0.000922,0.422012


In [312]:
offset = get_cg_offset(beam_offset)
print(f'{offset = }')


# Operations info
omega = 1 # rad/s
pitch_deg = 45
pitch_rad = np.deg2rad(pitch_deg)

r_hub = 1

offset = array([[0.1, 0. , 0. ],
       [0.1, 0. , 0. ]])


In [313]:
# Calculate inertial loads using the CoG accelerations

load_t7 = inertial_loads_fun_v04(pos,offset,M_mat_offset,r_hub,omega,pitch_rad)
load_t7 = np.reshape(load_t7, (-1,6))
print(f'{pos = }')

force, moment = np.split(load_t7,[3],axis=1)
print(f'{force = }')
print(f'{moment = }')


pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])
force = array([[ -3.23420616,   3.23420616, -66.66666667],
       [ -1.76579384,   1.76579384, -83.33333333]])
moment = array([[-0.78376975,  5.88289692,  1.10657386],
       [ 0.04956359,  8.38289692, -0.60657386]])


In [314]:
force_o = np.sum(force, axis=0)
force_o = b1_to_b2(force_o, pitch_rad,inv=True)
print(f'{force_o = }')

pos_o = pos[:,:3]
pos_o[:,2] += r_hub
print(f'{pos_o = }')

moment_o = np.sum(moment,axis=0) + np.cross(pos_o[0],force[0]) + np.cross(pos_o[1],force[1])
moment_o = b1_to_b2(moment_o, pitch_rad,inv=True)
print(f'{moment_o = }')

# print(np.sqrt(force_o[0]**2+force_o[1]**2))

force_o = array([-7.07106781e+00, -8.88178420e-16, -1.50000000e+02])
pos_o = array([[0., 0., 1.],
       [0., 0., 2.]])
moment_o = array([-1.06066017e+01,  7.99360578e-14,  5.00000000e-01])


In [315]:
f_analytical = np.array([-5*np.sqrt(2),0.,-150])
m_analytical = np.array([0.,0.,-f_analytical[0]*0.1*np.sqrt(2)/2])
# The M in the x-axis is not calculated


if np.allclose(force_o, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_o[1:], m_analytical[1:]): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_o, f_analytical) and np.allclose(moment_o[1:], m_analytical[1:]):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
The two results are the same.
Test passed.
